---
# Imports

In [1]:
# imports.py
from imports import *
from sklearn.datasets import make_blobs

# plotting magic
%matplotlib inline

# custom mods
import tidy
import model_h
import wrangle

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


---
# Wrangle

In [70]:
df=wrangle.wrangle_df()
# df.info()
# # model_h.print_btcusd(df)
# df.head()

cached csv


In [71]:
df['atr'] = talib.ATR(df.high, df.low, df.close, 14)

In [72]:
df=df.loc['2014-11-03':]

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2756 entries, 2014-11-03 to 2022-05-24
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   open                          2756 non-null   float64
 1   high                          2756 non-null   float64
 2   low                           2756 non-null   float64
 3   close                         2756 non-null   float64
 4   volume                        2756 non-null   float64
 5   fwd_log_ret                   2756 non-null   float64
 6   fwd_ret                       2756 non-null   float64
 7   fwd_pct_chg                   2756 non-null   float64
 8   fwd_close_positive            2756 non-null   float64
 9   macd                          2756 non-null   float64
 10  signal                        2756 non-null   float64
 11  histo                         2756 non-null   float64
 12  cross                         2756 non-null 

---
## Train / Validate Split

In [80]:
train_unscaled, val_unscaled=tidy.split_i(df)


# # train_ii, test=tidy.split_ii(df)
# # train_ii.info(),test.info()


---
## Scaler

In [75]:
transformer = RobustScaler().fit(train_unscaled)

In [76]:
train=transformer.transform(train_unscaled)
val=transformer.transform(val_unscaled)

In [83]:
train=pd.DataFrame(train,columns=train_unscaled.columns)
val=pd.DataFrame(val,columns=val_unscaled.columns)

In [ ]:
# def scaler_fit(X_train):
# 	for col in X_train.columns:
# 		if X_train[col].dtype == 'float64':
# 			xs=np.array(X_train[[col]]).reshape(-1, 1)
# 			X_train.loc[:,col]=RobustScaler().fit_transform(xs)
# 			print(col)
# 	return X_train


In [ ]:
# def scaler_tran(df):
# 	for col in df.columns:
# 		if df[col].dtype == 'float64':
# 			xs=np.array(df[[col]]).reshape(-1, 1)
# 			X_train.loc[:,col]=RobustScaler().transform(xs)
# 			print(col)
# 	return X_train


In [ ]:
# X_train=scaler_fit(X_train)


---
## Xy Split

In [89]:
X_train=train[['open', 'high', 'low', 'close', 'volume', 'obv_close_product','atr', 'macd', 'signal', 'histo', 'sentiment']]
X_val=val[['open', 'high', 'low', 'close', 'volume', 'obv_close_product','atr', 'macd', 'signal', 'histo', 'sentiment']]

In [92]:
y_train=train[[ 'fwd_ret']]
y_val=val[[ 'fwd_ret']]

---
# RFE Regression

In [100]:
lm = LinearRegression()

In [108]:
rfe = RFE(lm, n_features_to_select=8)

In [110]:
# Transforming data using RFE
X_rfe = rfe.fit_transform(X_train,y_train.fwd_ret)  
mask=rfe.support_
rfe_features = X_train.loc[:,mask].columns.tolist()

X_train=pd.DataFrame(X_rfe,columns=rfe_features)
X_train.head()


,open,high,low,close,volume,obv_close_product,atr,sentiment
0,-0.602299,-0.595441,-0.610061,-0.602894,-0.212896,-0.405592,-0.418585,1.599793
1,-0.602143,-0.595655,-0.610102,-0.602605,-0.212789,-0.434121,-0.419445,1.517606
2,-0.601797,-0.594543,-0.609534,-0.601722,-0.212624,-0.309356,-0.419485,1.532418
3,-0.600935,-0.593624,-0.608751,-0.600759,-0.212665,-0.168982,-0.419318,1.592087
4,-0.599917,-0.593646,-0.608412,-0.601434,-0.212742,-0.331815,-0.419602,1.655639


In [113]:
# var_ranks = rfe.ranking_
# var_names = X_rfe.columns.tolist()

# pd.DataFrame({'Var': var_names, 'Rank': var_ranks})


In [ ]:
# Transforming data using RFE
X_rfe = rfe.fit_transform(X_val,y_train.fwd_ret)  
mask=rfe.support_
rfe_features = X_train.loc[:,mask].columns.tolist()

X_train=pd.DataFrame(X_rfe,columns=rfe_features)
X_train.head()

In [114]:
X_val=X_val[rfe_features]

---
# Regression Model

In [118]:

line=lm.fit(X_train,y_train)


In [119]:

y_train['reg'] = lm.predict(X_train)


/var/folders/mk/w2hz4wxx7xn5hr_56wz536gh0000gn/T/ipykernel_36348/1534299188.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train['reg'] = lm.predict(X_train)


In [120]:

rmse_train = metrics.mean_squared_error(y_train.fwd_ret, y_train.reg)**(1/2)


In [121]:

y_val['reg'] = lm.predict(X_val)


/var/folders/mk/w2hz4wxx7xn5hr_56wz536gh0000gn/T/ipykernel_36348/2913920812.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_val['reg'] = lm.predict(X_val)


In [122]:

rmse_validate = metrics.mean_squared_error(y_val.fwd_ret, y_val.reg)**(1/2)

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)

RMSE for OLS using LinearRegression
Training/In-Sample:  5.829673054574351 
Validation/Out-of-Sample:  7.461529962262763


---
# *!!! Hard Hats Required Past This Point !!!*